In [6]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connect to database via CRUD Module
username = "aacuser"
password = "SNHU1234"

db = AnimalShelter()

# Load all data
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Logo (you must have an image file in the same directory as this script)
image_filename = 'Grazioso Salvare Logo.png'  # <<< Replace this with your real filename
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Rescue Dashboard - Created by Joshua Lee'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
    html.Hr(),

    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10
    ),
    
    html.H2('Created by Joshua Lee'),
    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# FILTER the data based on selected rescue type
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == "Water Rescue":
        query = {
            "$and": [
                {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == "Mountain or Wilderness Rescue":
        query = {
            "$and": [
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == "Disaster or Individual Tracking":
        query = {
            "$and": [
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
    else:  # Reset
        query = {}

    filtered_df = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')


# Update the pie chart based on table data
@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        dff = df
    else:
        dff = pd.DataFrame.from_dict(viewData)

    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]


# Highlight the selected column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]




# Update the map based on selected row
@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
               Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return None

    dff = pd.DataFrame.from_dict(viewData)
    selected_row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[selected_row]['location_lat'], dff.iloc[selected_row]['location_long']], children=[
                dl.Tooltip(dff.iloc[selected_row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_row]['name'])
                ])
            ])
        ])
    ]

app.run_server(debug=True)




✅ Connected to MongoDB successfully.
Dash app running on http://127.0.0.1:13749/
